In [1]:
from collections import namedtuple
from Crypto.Util.number import inverse
import hashlib

Point = namedtuple("Point", "x y")
O = 'INFINITY'

p = 9631668579539701602760432524602953084395033948174466686285759025897298205383
gx = 5664314881801362353989790109530444623032842167510027140490832957430741393367
gy = 3735011281298930501441332016708219762942193860515094934964869027614672869355
Ax = 3829488417236560785272607696709023677752676859512573328792921651640651429215
Ay = 7947434117984861166834877190207950006170738405923358235762824894524937052000
Bx = 9587224500151531060103223864145463144550060225196219072827570145340119297428
By = 2527809441042103520997737454058469252175392602635610992457770946515371529908


In [2]:
def is_on_curve(P):
    if P == O:
        return True
    else:
        return (P.y**2 - (P.x**3 + a*P.x + b)) % p == 0 and 0 <= P.x < p and 0 <= P.y < p

def point_inverse(P):
    if P == O:
        return P
    return Point(P.x, -P.y % p)

def point_addition(P, Q):
    if P == O:
        return Q
    elif Q == O:
        return P
    elif Q == point_inverse(P):
        return O
    else:
        if P == Q:
            s = (3*P.x**2 + a)*inverse(2*P.y, p) % p
        else:
            s = (Q.y - P.y) * inverse((Q.x - P.x), p) % p
    Rx = (s**2 - P.x - Q.x) % p
    Ry = (s*(P.x - Rx) - P.y) % p
    R = Point(Rx, Ry)
    assert is_on_curve(R)
    return R

def point_multiply(P, d):
    bits = bin(d)[2:]
    Q = O
    for bit in bits:
        Q = point_addition(Q, Q)
        if bit == '1':
            Q = point_addition(Q, P)
    assert is_on_curve(Q)
    return Q

In [3]:
a, b = var('a, b')
eq1 = (Ay**2 - (Ax**3 + a*Ax + b)).mod(p) == 0
eq2 = (By**2 - (Bx**3 + a*Bx + b)).mod(p) == 0
res = solve([eq1, eq2], a, b)
a = Integer(Mod(res[0][0].rhs().numerator(), p) / Mod(res[0][0].rhs().denominator(), p))
b = Integer(Mod(res[0][1].rhs().numerator(), p) / Mod(res[0][1].rhs().denominator(), p))
print(f'a = {a}')
print(f'b = {b}')

a = 9605275265879631008726467740646537125692167794341640822702313056611938432994
b = 7839838607707494463758049830515369383778931948114955676985180993569200375480


In [4]:
if (4*a**3 + 27*b**2) % p == 0:
    print('Singular!')

Singular!


In [5]:
# F.<x> = GF(p)[]
# dx = 3*x**2 + a
# x1 = Integer(dx.roots()[0][0])
# x2 = Integer(dx.roots()[1][0])

# print(f'x1 = {x1}')
# print(f'x2 = {x2}')
# if is_on_curve(Point(x1, 0)):
#     print('x1 is on curve!')
# if is_on_curve(Point(x2, 0)):
#     print('x2 is on curve!')

F.<x,y> = GF(p)[]
f = x**3 + a*x + b
G = Point(gx, gy)
A = Point(Ax, Ay)
B = Point(Bx, By)
print(f.factor())

# f_ = f.subs(x = x + x2)
# G_ = Point(G[0] - x2, G[1])
# A_ = Point(A[0] - x2, A[1])
# B_ = Point(B[0] - x2, B[1])
# print(f_.factor())

(x + 1706485822346415641443806104662801825943914230110363749830437374602647864828) * (x + 8778425668366493782038529472271552171423076833119284811370540338595974272969)^2


In [6]:
alpha = -8778425668366493782038529472271552171423076833119284811370540338595974272969
beta = -1706485822346415641443806104662801825943914230110363749830437374602647864828

t = GF(p)(alpha-beta).square_root() # get the (alpha-beta)^(1/2) % p
phi = (y + t*(x-alpha)) / (y-t*(x-alpha))

u = phi(A[0], A[1]) % p
v = phi(G[0], G[1]) % p
dA = discrete_log(u, v)
dA

1532487521612462894579517163606359285989568203515734083099567402780433190052

In [7]:
enc = bytes.fromhex('1536c5b019bd24ddf9fc50de28828f727190ff121b709a6c63c4f823ec31780ad30d219f07a8c419c7afcdce900b6e89b37b18b6daede22e5445eb98f3ca2e40')

k = point_multiply(B, dA).x
k = hashlib.sha512(str(k).encode('ascii')).digest()
flag = bytes(ci ^^ ki for ci, ki in zip(enc.ljust(len(k), b'\0'), k))
flag

b'FLAG{adbffefdb46a99fad0042dd3c10fdc414fadd25c}\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'